## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-28-17-48-44 +0000,wapo,New York Mayor Eric Adams ends reelection bid,https://www.washingtonpost.com/politics/2025/0...
1,2025-09-28-17-48-21 +0000,cbc,Canada Post denies Ottawa's reforms give it le...,https://www.cbc.ca/news/politics/canada-post-o...
2,2025-09-28-17-48-15 +0000,nyt,"Shooting at Mormon Church in Grand Blanc, Mich...",https://www.nytimes.com/live/2025/09/28/us/gra...
3,2025-09-28-17-44-49 +0000,nyt,‘One Battle After Another’ Ticket Sales Are So...,https://www.nytimes.com/2025/09/28/business/me...
4,2025-09-28-17-44-38 +0000,nypost,Eric Adams rips political ‘extremism’ as he dr...,https://nypost.com/2025/09/28/us-news/eric-ada...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2310/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
58,trump,29
49,dead,11
157,russia,9
18,shooting,9
0,new,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
31,2025-09-28-15-55-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...,80
28,2025-09-28-16-05-59 +0000,nypost,Protesters clash with ICE agents as Trump depl...,https://nypost.com/2025/09/28/us-news/proteste...,63
7,2025-09-28-17-42-00 +0000,wsj,Michigan Mormon Church Shooting Leaves at Leas...,https://www.wsj.com/us-news/michigan-mormon-ch...,63
147,2025-09-27-23-18-16 +0000,nyt,"Trump Says He Has Ordered Troops to Portland, ...",https://www.nytimes.com/2025/09/27/us/politics...,62
153,2025-09-27-22-19-00 +0000,wsj,"Trump Says He Is Ordering Troops to Portland, ...",https://www.wsj.com/us-news/trump-says-he-is-o...,62


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
31,80,2025-09-28-15-55-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...
7,63,2025-09-28-17-42-00 +0000,wsj,Michigan Mormon Church Shooting Leaves at Leas...,https://www.wsj.com/us-news/michigan-mormon-ch...
4,53,2025-09-28-17-44-38 +0000,nypost,Eric Adams rips political ‘extremism’ as he dr...,https://nypost.com/2025/09/28/us-news/eric-ada...
28,34,2025-09-28-16-05-59 +0000,nypost,Protesters clash with ICE agents as Trump depl...,https://nypost.com/2025/09/28/us-news/proteste...
103,31,2025-09-28-09-08-40 +0000,nypost,"Russia fires over 600 drones, missiles in dead...",https://nypost.com/2025/09/28/world-news/large...
183,31,2025-09-27-18-06-21 +0000,bbc,Israeli strikes kill dozens in Gaza including ...,https://www.bbc.com/news/articles/c87y58jgn5lo...
69,29,2025-09-28-13-13-51 +0000,bbc,Building of three new towns will start before ...,https://www.bbc.com/news/articles/cly1geen679o...
129,26,2025-09-28-03-29-21 +0000,nypost,Rep. Marjorie Taylor Greene warns in X rant t...,https://nypost.com/2025/09/27/us-news/rep-marj...
169,25,2025-09-27-19-50-39 +0000,nypost,US preps for attacks against drug lords deep i...,https://nypost.com/2025/09/27/us-news/us-preps...
36,24,2025-09-28-15-44-01 +0000,nypost,Accused Southport shooter Nigel Edge once esco...,https://nypost.com/2025/09/28/us-news/accused-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
